In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 100)
plt.style.use("default")

In [10]:
train_clf = pd.read_csv(r"C:\Users\sffra\Downloads\BSE 2025-2026\cml_final\data\raw\MIMIC III dataset HEF\mimic_train_HEF.csv")  # adjust path
test_clf  = pd.read_csv(r"C:\Users\sffra\Downloads\BSE 2025-2026\cml_final\data\raw\MIMIC III dataset HEF\mimic_test_HEF.csv") 

train_clf.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,DiasBP_Max,DiasBP_Mean,MeanBP_Min,MeanBP_Max,MeanBP_Mean,RespRate_Min,RespRate_Max,RespRate_Mean,TempC_Min,TempC_Max,TempC_Mean,SpO2_Min,SpO2_Max,SpO2_Mean,Glucose_Min,Glucose_Max,Glucose_Mean,GENDER,DOB,DOD,ADMITTIME,DISCHTIME,DEATHTIME,Diff,ADMISSION_TYPE,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ICD9_diagnosis,FIRST_CAREUNIT,LOS
0,0,55440,195768,228357,89.0,145.0,121.043478,74.0,127.0,106.586957,42.0,90.0,61.173913,59.0,94.0,74.543478,15.0,30.0,22.347826,35.111111,36.944444,36.080247,90.0,99.0,95.739130,111.0,230.0,160.777778,F,2108-07-16 00:00:00,2180-03-09 00:00:00,2178-02-06 10:35:00,2178-02-13 18:30:00,NaN,-61961.78470,EMERGENCY,Medicare,PROTESTANT QUAKER,SINGLE,WHITE,GASTROINTESTINAL BLEED,5789,MICU,4.5761
1,0,76908,126136,221004,63.0,110.0,79.117647,89.0,121.0,106.733333,49.0,74.0,64.733333,58.0,84.0,74.800000,13.0,21.0,16.058824,36.333333,36.611111,36.472222,98.0,100.0,99.058824,103.0,103.0,103.000000,F,2087-01-16 00:00:00,NaN,2129-02-12 22:34:00,2129-02-13 16:20:00,NaN,-43146.18378,EMERGENCY,Private,UNOBTAINABLE,MARRIED,WHITE,ESOPHAGEAL FOOD IMPACTION,53013,MICU,0.7582
2,0,95798,136645,296315,81.0,98.0,91.689655,88.0,138.0,112.785714,45.0,67.0,56.821429,64.0,88.0,72.888889,13.0,21.0,15.900000,36.444444,36.888889,36.666667,100.0,100.0,100.000000,132.0,346.0,217.636364,F,2057-09-17 00:00:00,NaN,2125-11-17 23:04:00,2125-12-05 17:55:00,NaN,-42009.96157,EMERGENCY,Medicare,PROTESTANT QUAKER,SEPARATED,BLACK/AFRICAN AMERICAN,UPPER GI BLEED,56983,MICU,3.7626
3,0,40708,102505,245557,76.0,128.0,98.857143,84.0,135.0,106.972973,30.0,89.0,41.864865,48.0,94.0,62.783784,12.0,35.0,26.771429,36.333333,39.500000,37.833333,78.0,100.0,95.085714,108.0,139.0,125.000000,F,2056-02-27 00:00:00,2132-03-01 00:00:00,2131-01-26 08:00:00,2131-02-05 16:23:00,NaN,-43585.37922,ELECTIVE,Medicare,NOT SPECIFIED,WIDOWED,WHITE,HIATAL HERNIA/SDA,5533,SICU,3.8734
4,0,28424,127337,225281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,137.0,113.000000,F,2066-12-19 00:00:00,2147-01-18 00:00:00,2146-05-04 02:02:00,2146-05-20 18:40:00,NaN,-50271.76602,EMERGENCY,Medicare,JEWISH,WIDOWED,WHITE,ABDOMINAL PAIN,56211,TSICU,5.8654


Cleaning

In [11]:
id_cols = ["icustay_id", "hadm_id", "subject_id"]

# Target vector (label)
y_train = train_clf["HOSPITAL_EXPIRE_FLAG"].copy()

# Keep ICU stay IDs for later (for final predictions)
train_ids = train_clf["icustay_id"].copy()  
test_ids  = test_clf["icustay_id"].copy()

# Columns that should NOT be used as features
leakage_cols = ["ADMITTIME", "ICD9_diagnosis", "DIAGNOSIS", "DOB", "DEATHTIME", "DISCHTIME", "DOD", "LOS", "HOSPITAL_EXPIRE_FLAG"]

# For safety, only drop columns that actually exist in the dataframe
cols_to_drop_train = [c for c in (id_cols + leakage_cols) if c in train_clf.columns]
cols_to_drop_test  = [c for c in id_cols if c in test_clf.columns]

# New, cleaned feature dataframes (raw features before any encoding/imputation)
X_train_raw = train_clf.drop(columns=cols_to_drop_train).copy()
X_test_raw  = test_clf.drop(columns=cols_to_drop_test).copy()


In [5]:
import numpy as np
from sklearn.impute import SimpleImputer

num_cols = X_train_raw.select_dtypes(include=[np.number]).columns.tolist()

num_imputer = SimpleImputer(strategy="median")

# 3. Fit on TRAIN numeric data only
num_imputer.fit(X_train_raw[num_cols])

# 4. Transform both TRAIN and TEST numeric data
X_train_num = num_imputer.transform(X_train_raw[num_cols])
X_test_num  = num_imputer.transform(X_test_raw[num_cols])

# 5. Put them back into DataFrames if you want to keep column names
X_train_imputed = X_train_raw.copy()
X_test_imputed  = X_test_raw.copy()

X_train_imputed[num_cols] = X_train_num
X_test_imputed[num_cols]  = X_test_num

In [12]:
cat_cols = X_train_imputed.select_dtypes(exclude=[np.number]).columns.tolist()

# 1. Compute the most frequent MARITAL_STATUS *from the training data only*
marital_mode = X_train_imputed["MARITAL_STATUS"].mode()[0]
print("Most frequent MARITAL_STATUS in train:", marital_mode)

# 2. Fill missing MARITAL_STATUS in TRAIN using that value
X_train_imputed["MARITAL_STATUS"] = (
    X_train_imputed["MARITAL_STATUS"].fillna(marital_mode)
)

# 3. Fill missing MARITAL_STATUS in TEST using the SAME value
X_test_imputed["MARITAL_STATUS"] = (
    X_test_imputed["MARITAL_STATUS"].fillna(marital_mode)
)

# 4. Sanity check: no missing values left in that column
print("Train MARITAL_STATUS NaNs:", X_train_imputed["MARITAL_STATUS"].isna().sum())
print("Test MARITAL_STATUS NaNs:",  X_test_imputed["MARITAL_STATUS"].isna().sum())


Most frequent MARITAL_STATUS in train: MARRIED
Train MARITAL_STATUS NaNs: 0
Test MARITAL_STATUS NaNs: 0


In [13]:
import numpy as np
import pandas as pd

# Conservative physiological lower bounds for *minimum* BPs
BP_MIN_LOWER_BOUNDS = {
    "SysBP_Min":  40.0,  # systolic
    "DiasBP_Min": 10.0,  # diastolic
    "MeanBP_Min": 30.0,  # mean arterial pressure
}

def clean_min_bp_outliers(df: pd.DataFrame,
                          lower_bounds: dict = BP_MIN_LOWER_BOUNDS) -> pd.DataFrame:
    """
    Replace implausibly low minimum blood pressures with NaN.

    This targets ICU summary columns like:
        SysBP_Min, DiasBP_Min, MeanBP_Min

    Any non-missing value < lower_bounds[col] is treated as an error/artifact
    and set to NaN, to be handled later by your imputer.
    """
    df = df.copy()

    for col, low in lower_bounds.items():
        if col not in df.columns:
            print(f"[clean_min_bp_outliers] Column {col} not found, skipping.")
            continue

        s = df[col]
        mask_valid = s.notna()
        n_valid = mask_valid.sum()
        if n_valid == 0:
            print(f"[clean_min_bp_outliers] Column {col} has no valid values, skipping.")
            continue

        below_mask = (s < low) & mask_valid
        n_below = below_mask.sum()

        if n_below > 0:
            pct_valid = n_below / n_valid * 100.0
            print(
                f"[clean_min_bp_outliers] {col}: "
                f"setting {n_below} values ({pct_valid:.3f}% of valid) "
                f"below {low} mmHg to NaN"
            )
            df.loc[below_mask, col] = np.nan
        else:
            print(f"[clean_min_bp_outliers] {col}: no values below {low} mmHg")

    return df

# --- Use this *before* add_engineered_features ---
X_train_raw = clean_min_bp_outliers(X_train_raw)
X_test_raw  = clean_min_bp_outliers(X_test_raw)


[clean_min_bp_outliers] SysBP_Min: setting 99 values (0.530% of valid) below 40.0 mmHg to NaN
[clean_min_bp_outliers] DiasBP_Min: setting 6 values (0.032% of valid) below 10.0 mmHg to NaN
[clean_min_bp_outliers] MeanBP_Min: setting 797 values (4.262% of valid) below 30.0 mmHg to NaN
[clean_min_bp_outliers] SysBP_Min: setting 31 values (0.664% of valid) below 40.0 mmHg to NaN
[clean_min_bp_outliers] DiasBP_Min: setting 1 values (0.021% of valid) below 10.0 mmHg to NaN
[clean_min_bp_outliers] MeanBP_Min: setting 227 values (4.857% of valid) below 30.0 mmHg to NaN


## 4. Feature engineering (v2)

In this version we create additional features from the raw columns
(e.g. age at admission, time-of-day information, simple derived
variables) **before** applying the preprocessing pipeline.
All engineered features are computed row-wise without using the
target or information from other patients to avoid data leakage.


In [14]:
import numpy as np

def add_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add row-level numeric features:
    - Ranges (max - min) for vitals
    - Abnormal vital flags (0/1)
    - instability_count = sum of selected flags
    - Nonlinear/composite features (shock index, pulse pressure, deviations, interactions)
    """
    df = df.copy()
    eps = 1e-3  # avoid divide-by-zero

    # -----------------------------
    # 1. RANGES (instability of vitals)
    # -----------------------------
    df["HR_range"]        = df["HeartRate_Max"]  - df["HeartRate_Min"]
    df["SysBP_range"]     = df["SysBP_Max"]      - df["SysBP_Min"]
    df["DiasBP_range"]    = df["DiasBP_Max"]     - df["DiasBP_Min"]
    df["MeanBP_range"]    = df["MeanBP_Max"]     - df["MeanBP_Min"]
    df["RespRate_range"]  = df["RespRate_Max"]   - df["RespRate_Min"]
    df["TempC_range"]     = df["TempC_Max"]      - df["TempC_Min"]
    df["SpO2_range"]      = df["SpO2_Max"]       - df["SpO2_Min"]
    df["Glucose_range"]   = df["Glucose_Max"]    - df["Glucose_Min"]

    # -----------------------------
    # 2. ABNORMAL VITAL FLAGS (0/1)
    # -----------------------------

    # Cardiovascular
    df["tachy_flag"]       = (df["HeartRate_Max"] >= 120).astype(int)
    df["hypotension_flag"] = (
        (df["SysBP_Min"] < 90) | (df["MeanBP_Min"] < 65)
    ).astype(int)

    # Respiratory
    df["tachypnea_flag"]   = (df["RespRate_Max"] >= 30).astype(int)

    # Temperature
    df["fever_flag"]       = (df["TempC_Max"] >= 38.0).astype(int)

    # Oxygenation
    df["hypoxia_flag"]     = (df["SpO2_Min"] < 92).astype(int)

    # Glucose
    df["hyperglycemia_flag"] = (df["Glucose_Max"] >= 200).astype(int)

    # -----------------------------
    # 3. INSTABILITY COUNT
    # -----------------------------
    instability_flags = [
        "tachy_flag",
        "hypotension_flag",
        "tachypnea_flag",
        "fever_flag",
        "hypoxia_flag",
        "hyperglycemia_flag",
    ]
    df["instability_count"] = df[instability_flags].sum(axis=1)

    # -----------------------------
    # 4. NONLINEAR / COMPOSITE FEATURES
    # -----------------------------

    # 4.1 Shock index (higher is worse): HR / SBP
    df["shock_index_mean"] = df["HeartRate_Mean"] / (df["SysBP_Mean"].clip(lower=eps))
    df["shock_index_min"]  = df["HeartRate_Min"]  / (df["SysBP_Min"].clip(lower=eps))

    # 4.2 Pulse pressure (SBP - DBP)
    df["pulse_pressure_mean"] = df["SysBP_Mean"] - df["DiasBP_Mean"]
    df["pulse_pressure_min"]  = df["SysBP_Min"]  - df["DiasBP_Min"]
    df["pulse_pressure_max"]  = df["SysBP_Max"]  - df["DiasBP_Max"]

    # 4.3 Distance from "normal" temperature (symmetric)
    df["temp_dev_mean"] = (df["TempC_Mean"] - 37.0).abs()

    # 4.4 How far SpO2 fell below 92 (0 if never below)
    df["spo2_deficit"] = (92.0 - df["SpO2_Min"]).clip(lower=0.0)

    # 4.5 How far glucose went above 140 (0 if always <= 140)
    df["glucose_excess"] = (df["Glucose_Max"] - 140.0).clip(lower=0.0)

    # 4.6 Interaction flags: "both bad" situations
    df["tachy_and_hypotension"] = (
        (df["tachy_flag"] == 1) & (df["hypotension_flag"] == 1)
    ).astype(int)

    df["hypoxia_and_tachypnea"] = (
        (df["hypoxia_flag"] == 1) & (df["tachypnea_flag"] == 1)
    ).astype(int)

    return df


# Apply feature engineering to TRAIN and TEST
X_train_raw = add_engineered_features(X_train_raw)
X_test_raw  = add_engineered_features(X_test_raw)

print("X_train_raw shape after FE:", X_train_raw.shape)
print("X_test_raw shape after FE:", X_test_raw.shape)


X_train_raw shape after FE: (20885, 57)
X_test_raw shape after FE: (5221, 61)


## 4. Preprocessing pipeline & train/validation split (classification)

In this section we:

- Use the raw feature matrix `X_train_raw`
- Define numerical and categorical feature groups.
- Build a scikit-learn `ColumnTransformer` that:
  - imputes missing values (median for numeric, most frequent for categorical),
  - scales numeric features for linear models,
  - one-hot encodes categorical features.
- Create a stratified train/validation split on the classification target `HOSPITAL_EXPIRE_FLAG`.

This preprocessing will be shared by all models in this notebook.


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# -----------------------------
# 1. Define feature matrix and target for CLASSIFICATION
# -----------------------------

# Use the raw feature matrix AFTER dropping IDs + leakage
X = X_train_raw.copy()
y = y_train.copy()   # HOSPITAL_EXPIRE_FLAG from earlier cell

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)
print("Positive rate:", y.mean().round(3))

# -----------------------------
# 2. Ensure numeric / categorical feature lists
# -----------------------------

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numeric features ({len(num_cols)}):", num_cols[:10], "...")
print(f"Categorical features ({len(cat_cols)}):", cat_cols)

# -----------------------------
# 3. Define preprocessing for numeric & categorical columns
# -----------------------------

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

# -----------------------------
# 4. Create train/validation split (stratified)
# -----------------------------

X_train, X_valid, y_train_split, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

print("Train size:", X_train.shape[0], "Validation size:", X_valid.shape[0])
print("Train positive rate:", y_train_split.mean().round(3))
print("Valid positive rate:", y_valid.mean().round(3))


Feature matrix shape: (20885, 57)
Target shape: (20885,)
Positive rate: 0.112
Numeric features (50): ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean', 'MeanBP_Min'] ...
Categorical features (7): ['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT']
Train size: 16708 Validation size: 4177
Train positive rate: 0.112
Valid positive rate: 0.112


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

# -----------------------------
# 1. Define base classifiers
# -----------------------------
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
    solver="lbfgs",
)

rf_clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42,
)

hgb_clf = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=None,
    max_leaf_nodes=31,
    random_state=42,
)

# -----------------------------
# 2. Wrap each in a pipeline (preprocessor + model)
# -----------------------------
log_reg_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", log_reg),
    ]
)

rf_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", rf_clf),
    ]
)

hgb_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", hgb_clf),
    ]
)

# -----------------------------
# 3. Fit and evaluate each baseline on the hold-out validation set
# -----------------------------
def evaluate_model(name, pipe):
    pipe.fit(X_train, y_train_split)
    y_valid_proba = pipe.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_valid_proba)
    print(f"{name} ROC-AUC on validation: {auc:.4f}")
    return auc

auc_logreg = evaluate_model("Logistic Regression", log_reg_pipe)
auc_rf     = evaluate_model("Random Forest", rf_pipe)
auc_hgb    = evaluate_model("HistGradientBoosting", hgb_pipe)


Logistic Regression ROC-AUC on validation: 0.8082
Random Forest ROC-AUC on validation: 0.8096
HistGradientBoosting ROC-AUC on validation: 0.8279


In [17]:
from sklearn.ensemble import VotingClassifier

# Base estimators for the ensemble (just the classifiers, not pipelines)
estimators = [
    ("lr", log_reg),
    ("rf", rf_clf),
    ("hgb", hgb_clf),
]

voting_clf = VotingClassifier(
    estimators=estimators,
    voting="soft",          # average predicted probabilities
    n_jobs=-1
)

voting_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", voting_clf),
    ]
)

voting_auc = evaluate_model("Soft Voting Ensemble", voting_pipe)


Soft Voting Ensemble ROC-AUC on validation: 0.8267


In [18]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=estimators,               # same base models as above
    final_estimator=LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ),
    cv=5,                                # internal CV to create out-of-fold preds
    n_jobs=-1,
    stack_method="auto",
)

stacking_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", stacking_clf),
    ]
)

stacking_auc = evaluate_model("Stacking Ensemble", stacking_pipe)


Stacking Ensemble ROC-AUC on validation: 0.8283
